# Import 📚

## Import library

In [ ]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
import openai
import pandas as pd
import json
from sklearn.model_selection import train_test_split

## Import training Data

In [ ]:
url = 'https://raw.githubusercontent.com/vgentile98/text_difficulty_prediction/main/data/training_data.csv'

df= pd.read_csv(url)

df

,id,sentence,difficulty
0,0,Les coûts kilométriques réels peuvent diverger...,C1
1,1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,2,Le test de niveau en français est sur le site ...,A1
3,3,Est-ce que ton mari est aussi de Boston?,A1
4,4,"Dans les écoles de commerce, dans les couloirs...",B1
...,...,...,...
4795,4795,"C'est pourquoi, il décida de remplacer les hab...",B2
4796,4796,Il avait une de ces pâleurs splendides qui don...,C1
4797,4797,"Et le premier samedi de chaque mois, venez ren...",A2
4798,4798,Les coûts liés à la journalisation n'étant pas...,C2


# Creation of the Da Vinci fine-tune model 🤖

In [ ]:
# Prepare the data for the fine tuning
# The data need to be in this format {'prompt': ..., 'completion':...}
data_for_tuning = [
    {"prompt": row["sentence"], "completion": f" {row['difficulty']}"}
    for index, row in df.iterrows()
]

In [ ]:
data_for_tuning[0]

{'prompt': "Les coûts kilométriques réels peuvent diverger sensiblement des valeurs moyennes en fonction du moyen de transport utilisé, du taux d'occupation ou du taux de remplissage, de l'infrastructure utilisée, de la topographie des lignes, du flux de trafic, etc.",
 'completion': ' C1'}

In [ ]:
#Split the training data to enable the fine tuning job to have feedback

train_data, val_data = train_test_split(
    data_for_tuning,
    test_size=0.1,
    random_state=42
)

In [ ]:
# Create a fonction that transform the data in the correct JSONl format asked by OpenAI
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [ ]:
client = openai.OpenAI(api_key = 'YOUR API KEY')

In [ ]:
# Give the data to OpenAi and it return the file id that we will use to fine tune the model
training_file = client.files.create(
    file=open("train.jsonl", "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open("val.jsonl", "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

NameError: name 'client' is not defined

In [ ]:
training_file_id = 'YOUR TRAINING FILE ID'
validation_file_id ='YOUR TEST FILE ID'

In [ ]:
#Creation of the fine tuning job, we use 'davinci-002' but you can use 'gpt-3.5' or other OpenAI model
response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="davinci-002"

)
response

FineTuningJob(id='ftjob-YUuXc7NtfltRYgbhY39MK1yI', created_at=1715767092, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='davinci-002', object='fine_tuning.job', organization_id='org-Wh1zmfMydytSG9b28oop33Pe', result_files=[], seed=505317890, status='validating_files', trained_tokens=None, training_file='file-beupkSoatXbQIqULVuhf4no3', validation_file='file-o74sgEHpQKahPH8gf4XHxZKd', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
job_id = 'YOUR JOB ID'

In [ ]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-YUuXc7NtfltRYgbhY39MK1yI', created_at=1715767092, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:personal::9P5qjeS4', finished_at=1715767712, hyperparameters=Hyperparameters(n_epochs=3, batch_size=8, learning_rate_multiplier=16), model='davinci-002', object='fine_tuning.job', organization_id='org-Wh1zmfMydytSG9b28oop33Pe', result_files=['file-d5o25rw4tDJCehKZIq0U8NSD'], seed=505317890, status='succeeded', trained_tokens=425631, training_file='file-beupkSoatXbQIqULVuhf4no3', validation_file='file-o74sgEHpQKahPH8gf4XHxZKd', estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-wUauQ0iMGeuyCN4yc1XV7aoS', created_at=1715700168, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:davinci-002:personal::9OoRnpw6', finished_at=1715700818, hyperparameters=Hyperparameters(n_epochs=3, batch_size=9, learning_rate_multiplier=16), model='davinci-002

In [ ]:
model_id = 'ft:davinci-002:personal::9P5qjeS4'

# Testing the model 🧪

##Import test data

In [ ]:
url_test = 'https://raw.githubusercontent.com/vgentile98/text_difficulty_prediction/main/data/unlabelled_test_data.csv'

df_test = pd.read_csv(url_test)

df_test.head()

,id,sentence
0,0,Nous dûmes nous excuser des propos que nous eû...
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...
2,2,"Et, paradoxalement, boire froid n'est pas la b..."
3,3,"Ce n'est pas étonnant, car c'est une saison my..."
4,4,"Le corps de Golo lui-même, d'une essence aussi..."


## Generate prediction

In [ ]:
def generate_predictions(df, model_id):
    # Add a new column to the DataFrame to store predicted difficulties
    df['difficulty'] = None

    # Iterate on each line of the DataFrame
    for index, row in df.iterrows():
        # Creating a request to the model
        response = client.completions.create(
            model=model_id,
            prompt=row['sentence'],
            max_tokens=5,
            n=1, temperature= 0.2 ,seed=42
        )

        # Save the model response in the DataFrame
        df.at[index, 'difficulty'] = response.choices[0].text.strip()

    return df

In [ ]:
#Use the fonction to generate prediction
df_test = generate_predictions(df_test, model_id)

df_test

,id,sentence,difficulty
0,0,Nous dûmes nous excuser des propos que nous eû...,C2 C2 C
1,1,Vous ne pouvez pas savoir le plaisir que j'ai ...,A2 A2 A
2,2,"Et, paradoxalement, boire froid n'est pas la b...",B2 B1 A
3,3,"Ce n'est pas étonnant, car c'est une saison my...",A2 B1 A
4,4,"Le corps de Golo lui-même, d'une essence aussi...",C2 C2 C
...,...,...,...
1195,1195,C'est un phénomène qui trouve une accélération...,B1 B1 B
1196,1196,Je vais parler au serveur et voir si on peut d...,A2 A2 A
1197,1197,Il n'était pas comme tant de gens qui par pare...,C2 C2 C
1198,1198,Ils deviennent dangereux pour notre économie.,B2 B2 B


In [ ]:
df_submission = df_test.loc[:, ['id', 'difficulty']]
df_submission

,id,difficulty
0,0,C2 C2 C
1,1,A2 A2 A
2,2,B2 B1 A
3,3,A2 B1 A
4,4,C2 C2 C
...,...,...
1195,1195,B1 B1 B
1196,1196,A2 A2 A
1197,1197,C2 C2 C
1198,1198,B2 B2 B


In [ ]:
#Take only the first token of prediction
df_submission['difficulty'] = df_submission['difficulty'].apply(lambda x: x[:2])
df_submission

,id,difficulty
0,0,C2
1,1,A2
2,2,B2
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,B2


In [ ]:
df_submission.to_csv('submission.csv', index=False)

# Vérification du contenu du fichier CSV créé
with open('submission.csv', 'r') as file:
    print(file.read())

id,difficulty
0,C2
1,A2
2,B2
3,A2
4,C2
5,C2
6,A2
7,A2
8,C2
9,B1
10,A1
11,A2
12,B2
13,C1
14,A1
15,A2
16,B2
17,A1
18,B1
19,A1
20,C2
21,B2
22,C1
23,C1
24,B1
25,C2
26,A1
27,A1
28,C2
29,B2
30,A2
31,B1
32,A1
33,A2
34,A2
35,A2
36,C1
37,B1
38,A1
39,A1
40,B1
41,B2
42,C2
43,C1
44,C1
45,C1
46,C1
47,C2
48,A1
49,A2
50,C1
51,A1
52,B2
53,A1
54,A1
55,C1
56,C1
57,B2
58,C1
59,A2
60,B2
61,A2
62,C1
63,C1
64,C1
65,B2
66,B2
67,A2
68,B2
69,C1
70,A1
71,B2
72,B2
73,B1
74,B2
75,A2
76,A2
77,C1
78,B1
79,B2
80,A2
81,A2
82,B2
83,B1
84,A2
85,B2
86,B1
87,B1
88,C1
89,B1
90,C2
91,A2
92,C1
93,C1
94,B1
95,A2
96,A2
97,A2
98,B1
99,C2
100,C1
101,C1
102,A2
103,A2
104,B1
105,C1
106,B1
107,A1
108,B1
109,C1
110,C2
111,A1
112,C1
113,C1
114,C2
115,B2
116,B2
117,A2
118,A2
119,C1
120,A2
121,B2
122,B1
123,C1
124,A2
125,B2
126,C1
127,C1
128,C1
129,B2
130,A2
131,A2
132,C1
133,C2
134,C2
135,C2
136,A2
137,B2
138,A2
139,B1
140,A1
141,C1
142,C2
143,C2
144,C1
145,B2
146,A1
147,C1
148,B2
149,B1
150,B2
151,B1
152,A2
153,A2
154,A1
155,B2
156,